In [ ]:
#定义类别,文件名
category = {'a1': '生活费',
            'a2': '零花钱',
            'a3': '工资',
            'b1': '学习用品',
            'b2': '日用品',
            'b3': '衣服鞋子',
            'b4': '食品', }
fileName = 'details.txt'
#写入表头
with open(fileName, encoding='utf-8') as fp:
    text=fp.readlines()
    if '类别编码,发生日期,金额,备注\n' not in text:
        with open(fileName, 'w+', encoding='utf-8') as fp:
            fp.write('类别编码,发生日期,金额,备注\n')
# 收支明细：类别编码、发生日期、金额、备注
class detail():
    def __init__(self, itemStr):
        #收支明细字符串形式
        self.itemStr = itemStr
        #类别编号
        self.cateCode = None
        #类别名称
        self.cateName = None
        #年
        self.dateYear = None
        #月
        self.dateMonth = None
        #日
        self.dateDay = None
        #金额
        self.money = None
        #备注
        self.note = None
        #初始化参数
        self.initProp()

    def initProp(self):

        itemList = self.itemStr.split(',')
        self.cateCode = itemList[0]
        self.cateName = category[self.cateCode]
        self.dateYear = int(itemList[1].split('-')[0])
        self.dateMonth = int(itemList[1].split('-')[1])
        self.dateDay = int(itemList[1].split('-')[2])
        self.money = float(itemList[2])
        self.note = itemList[3]

    #输出明细字符串
    def printItemStr(self):
        print(self.itemStr)
    #将明细字符串录入到文件中
    def writeItemStr(self, fileName):
        fp = open(fileName, 'a', encoding='utf-8')
        fp.write(self.itemStr)
        fp.write('\n')
        fp.close()

#从文件中读入明细
def getDetailsFromFile(fileName):
    details = []
    fp = open(fileName, 'r', encoding='utf-8')
    for line in fp.readlines()[1:]:
        item = detail(line.replace('\n', ''))
        details.append(item)
    fp.close()
    return details

#录入明细
def entryDetails():
    categoryList = sorted(category.items())
    print('类别展示：\n\t' + ','.join([k + '-' + v for k, v in categoryList]))
    print('--->请逐笔输⼊类别编码、发⽣⽇期、⾦额、备注（各数据⽤英⽂逗号分隔,直接输⼊回⻋表示该条明细输⼊结束，输入end回到主界面），')
    print('\t例：a1,2020-1-26,2000.0,1⽉⽣活费')
    itemStr = input('请输⼊收⽀明细：')
    while itemStr != 'end':
        try:
            item = detail(itemStr)
            item.writeItemStr(fileName)
        except:
            print('输入格式不正确，格式为类别编码、发⽣⽇期、⾦额、备注，且数据为英文逗号隔开')
        itemStr = input('请输⼊收⽀明细：')

#根据月份统计输出
def displayByMonth():
    print('根据月份统计并输出⽤户各收⽀类别的汇总情况，输入格式为年-月，输入end回到主界面.')
    inp = input('请输⼊对收⽀类别数据进⾏汇总的⽉份：')

    details = getDetailsFromFile(fileName)

    if len(details) != 0:
        while inp != 'end':
            try:
                year = int(inp.split('-')[0])
                month = int(inp.split('-')[1])
                curDetails = [i for i in details if i.dateYear == year and i.dateMonth == month]
                results = {}
                if len(curDetails) != 0:
                    for item in curDetails:
                        if item.cateCode not in results:
                            results[item.cateCode] = item.money
                        else:
                            results[item.cateCode] += item.money
                    print('{}年{}⽉收⽀类别数据的汇总'.format(year, month))
                    print('收⼊/⽀出\t明细类别\t⾦额')
                    for rk, rv in results.items():
                        print('{}\t{}\t{}'.format('收入' if 'a' in rk else '支出', category[rk], rv))
                    print('{}年{}的总收入为{}，总支出为{}'.format(year, month,
                                                       sum([v for k, v in results.items() if 'a' in k]),
                                                       sum([v for k, v in results.items() if 'b' in k])))
                    isPrintDatails = input('是否输出该⽉的各笔明细（y为输出，其他为不输出）：')
                    if isPrintDatails == 'y':
                        curDetails = sorted(curDetails, key=lambda item: item.cateCode)
                        print('类别\t收⼊/⽀出\t发⽣⽇期\t⾦额\t备注')
                        for item in curDetails:
                            print('{}\t{}\t{}-{}-{}\t{}\t{}'.format(item.cateName,
                                                                    '收入' if 'a' in item.cateCode else '支出',
                                                                    item.dateYear, item.dateMonth, item.dateDay,
                                                                    item.money,
                                                                    item.note))

                else:
                    print('该月无收支明细数据！')
            except:
                print('输入格式不正确，格式为年-月')
            inp = input('请输⼊对收⽀类别数据进⾏汇总的⽉份,输入end回到主界面')

    else:
        print('暂无录入的收支明细，请先录入！')

#根据类别统计输出
def displayByCate():
    print('根据类别统计并输出⽤户各月份收⽀的汇总情况，输入格式为类别编号，输入end回到主界面.')
    inp = input('请输⼊对各月份收⽀数据进⾏汇总的类别编号：')

    details = getDetailsFromFile(fileName)

    if len(details) != 0:
        while inp != 'end':
            if inp in category:
                curDetails = [i for i in details if i.cateCode == inp]
                results = {}
                if len(curDetails) != 0:
                    for item in curDetails:
                        ym = '{}-{}'.format(item.dateYear, item.dateMonth)
                        if ym not in results:
                            results[ym] = item.money
                        else:
                            results[ym] += item.money
                    print('{}类各⽉收⽀类别数据的汇总'.format(inp))
                    print('收⼊/⽀出\t月份\t⾦额')
                    results = sorted(results.items(), key=lambda x: x[0])
                    for rk, rv in results:
                        print('{}\t{}\t{}'.format('收入' if 'a' in inp else '支出', rk, rv))
                    print('类别{}的总收入为{}，总支出为{}'.format(inp,
                                                       sum([v for k, v in results if 'a' in inp]),
                                                       sum([v for k, v in results if 'b' in inp])))
                    isPrintDatails = input('是否输出该类别的各笔明细（y为输出，其他为不输出）：')
                    if isPrintDatails == 'y':
                        curDetails = sorted(curDetails, key=lambda item: (item.dateYear, item.dateMonth, item.dateDay))
                        print('类别\t收⼊/⽀出\t发⽣⽇期\t⾦额\t备注')
                        for item in curDetails:
                            print('{}\t{}\t{}-{}-{}\t{}\t{}'.format(item.cateName,
                                                                    '收入' if 'a' in item.cateCode else '支出',
                                                                    item.dateYear, item.dateMonth, item.dateDay,
                                                                    item.money,
                                                                    item.note))

                else:
                    print('该类别无收支明细数据！')
            else:
                print('输入格式不正确，格式为类别编号')
            inp = input('请输⼊对各月份收⽀数据进⾏汇总的类别编号,输入end回到主界面')

    else:
        print('暂无录入的收支明细，请先录入！')


def main():
    sysText = '欢迎来到个⼈收⽀管理系统 !\n' \
              '输入以下序号进入对应操作\n' \
              '1 录⼊收⽀明细\n' \
              '2 按月统计各收⽀类别的汇总情况\n' \
              '3 按类别展示收支明细的汇总情况\n' \
              '4 退出系统\n'
    while 1:
        choose = None
        try:
            choose = int(input(sysText))
            assert choose > 0 and choose < 5
        except:
            print('输入有误')
        if choose == 1:
            entryDetails()
        elif choose == 2:
            displayByMonth()
        elif choose == 3:
            displayByCate()
        elif choose == 4:
            print('拜拜啦')
            break


main()
